In [1]:
import pandas as pd
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 200)

In [2]:
all_muts_df = pd.read_pickle("../data/ava/4_10_with_uniq_midpts.pkl")
all_muts_df.head()

,index,exp,ale,flask,isolate,tech_rep,presence,Position,Mutation Type,Sequence Change,Details,mutation target annotation,Reference Seq,sample,coding,range,gene RegulonDB ID,genetic features,oriC,pseudogene,TFBS,promoter,RBS,attenuator terminator,terminator,genetic,genomic features,genetic feature links,operons,operon links,regulators,regulator links,pathways,pathway links,COGs,COG links,imodulons,imodulon links,EC numbers,EC number links,products,product links,temperature,carbon-source,supplement,strain-description,taxonomy-id,base-media,nitrogen-source,phosphorous-source,sulfur-source,calcium-source
0,9,42C,1,124,1,1,1.0,702352,DEL,Δ21 bp,coding (380‑400/1149 nt),nagA,NaN,1 124 1 1,True,"(702352, 702372)",{ECK120000625},"[{'name': 'nagA', 'RegulonDB ID': 'ECK12000062...",False,False,{},{},{},{},{},True,"[{'name': 'nagA', 'RegulonDB ID': 'ECK12000062...",{'ECK120000625': ['ECK120000625']},"[{'name': 'nagBAC-umpH', 'RegulonDB ID': 'ECK1...",{'ECK125162049': ['ECK120000625']},"[{'name': 'PhoP', 'significantly associated co...","{'NagC': ['ECK125162049'], 'PhoP': ['ECK125162...","[{'name': 'Galactose metabolism', 'significant...","{'Galactose metabolism': ['ECK125162049'], 'Am...",[{'name': 'Carbohydrate transport and metaboli...,{'Carbohydrate transport and metabolism': ['EC...,"[{'name': 'NagC/TyrR', 'significantly associat...",{'NagC/TyrR': ['nagBAC-umpH']},"[{'name': '3.5.1.25', 'significantly associate...","{'3.5.1.25': ['ECK125162049', 'ECK125162049']}",[{'name': 'N-acetylglucosamine-6-phosphate dea...,{'N-acetylglucosamine-6-phosphate deacetylase ...,42 celsius,glucose(4),NaCl(0.5g/L) trace elements,WT,511145,M9,NH4Cl(1),KH2PO4(3) Na2HPO4(6.8),MgSO4(0.24),CaCl2(0.1)
1,10,42C,1,124,1,1,1.0,1308318,SNP,G→C,R110G (CGT→GGT),clsA,NaN,1 124 1 1,True,"(1308318, 1308318)",{ECK120001556},"[{'name': 'clsA', 'RegulonDB ID': 'ECK12000155...",False,False,{},{},{},{},{},True,"[{'name': 'clsA', 'RegulonDB ID': 'ECK12000155...",{'ECK120001556': ['ECK120001556']},"[{'name': 'clsA-yciU', 'RegulonDB ID': 'ECK125...",{'ECK125147415': ['ECK120001556']},[],{},[],{},[{'name': 'Lipid transport and metabolism'}],{'Lipid transport and metabolism': ['ECK125147...,[],{},[],{},[],{},42 celsius,glucose(4),NaCl(0.5g/L) trace elements,WT,511145,M9,NH4Cl(1),KH2PO4(3) Na2HPO4(6.8),MgSO4(0.24),CaCl2(0.1)
2,11,42C,1,124,1,1,1.0,3815859,DEL,Δ82 bp,None,rph,NaN,1 124 1 1,True,"(3815859, 3815940)",{ECK120000854},"[{'name': 'rph', 'RegulonDB ID': 'ECK120000854...",False,False,{},{},{},{ECK125144791},{},True,"[{'name': 'rph-pyrE attenuator terminator', 'R...","{'ECK120000854': ['ECK120000854'], 'ECK1200007...","[{'name': 'rph-pyrE', 'RegulonDB ID': 'ECK1200...","{'ECK120014627': ['ECK120000854', 'ECK12000079...",[],{},"[{'name': 'Pyrimidine metabolism', 'significan...","{'Pyrimidine metabolism': ['ECK120014627'], 'D...",[{'name': 'Nucleotide transport and metabolism'}],{'Nucleotide transport and metabolism': ['ECK1...,"[{'name': 'purR-KO', 'significantly associated...","{'purR-KO': ['rph-pyrE', 'rph-pyrE']}","[{'name': '2.4.2.10', 'significantly associate...","{'2.4.2.10': ['ECK120014627', 'ECK120014627']}",[{'name': 'Orotate phosphoribosyltransferase (...,{'Orotate phosphoribosyltransferase (EC 2.4.2....,42 celsius,glucose(4),NaCl(0.5g/L) trace elements,WT,511145,M9,NH4Cl(1),KH2PO4(3) Na2HPO4(6.8),MgSO4(0.24),CaCl2(0.1)
3,12,42C,1,124,1,1,1.0,4187550,SNP,C→T,A734V (GCG→GTG),rpoC,NaN,1 124 1 1,True,"(4187550, 4187550)",{ECK120000886},"[{'name': 'rpoC', 'RegulonDB ID': 'ECK12000088...",False,False,{},{},{},{},{},True,"[{'name': 'rpoC', 'RegulonDB ID': 'ECK12000088...",{'ECK120000886': ['ECK120000886']},"[{'name': 'rplKAJL-rpoBC', 'RegulonDB ID': 'EC...",{'ECK120016992': ['ECK120000886']},[],{},"[{'name': 'Purine metabolism', 'significantly ...","{'Purine metabolism': ['ECK120016992'], 'Pyrim...",[{'name': 'Transcription'}],{'Transcription': ['ECK120016992']},"[{'name': 'translation', 'significantly associ...",{'translation': ['rplKAJL-rpoBC']},"[{'name': '2.7.7.6'

In [3]:
targ_mut_df = pd.DataFrame()
FEAT_TYPES = ["genomic features", "operons", "pathways", "regulators"]  # ignoring genetic features for now since the genes get double with genomic features

for _, m in all_muts_df.iterrows():
    mut_assoc = False
    for ft in FEAT_TYPES:
        for f in m[ft]:
            if "glycerol(2.5)" in f["significantly associated conditions"]:
                mut_assoc = True
                break
        if mut_assoc:
            break
    
    if mut_assoc:
        targ_mut_df = targ_mut_df.append(m)

# display(len(targ_mut_df),targ_mut_df.head())

In [4]:
geno_feat_tot_mut_cnts = dict()
gen_feat_tot_mut_cnts = dict()
op_tot_mut_cnts = dict()
pwy_tot_mut_cnts = dict()
reg_tot_mut_cnts = dict()
imod_tot_mut_cnts = dict()
# Using the individual feature columns rather than link columns because this guarantees considering all linked and non-linked features
# for each of these levels, except genomic features.
for _, m in targ_mut_df.iterrows():
    
    for geno_feat in m["genomic features"]:
        geno_feat_annot = geno_feat["RegulonDB ID"] #+ ' ' + r["Mutation Type"]
        if geno_feat_annot not in geno_feat_tot_mut_cnts.keys():
            geno_feat_tot_mut_cnts[geno_feat_annot] = 0
        geno_feat_tot_mut_cnts[geno_feat_annot] += 1

    for gen_feat, links in m["genetic feature links"].items():
        if gen_feat not in gen_feat_tot_mut_cnts.keys():
            gen_feat_tot_mut_cnts[gen_feat] = 0
        gen_feat_tot_mut_cnts[gen_feat] += len(links)

    for op, links in m["operon links"].items():
        if op not in op_tot_mut_cnts.keys():
            op_tot_mut_cnts[op] = 0
        op_tot_mut_cnts[op] += len(links)
        
    for reg, links in m["regulator links"].items():        
        if reg not in reg_tot_mut_cnts.keys():
            reg_tot_mut_cnts[reg] = 0
        reg_tot_mut_cnts[reg] += len(links)

In [5]:
FEAT_COLS = [
    "genomic features",
    "operons",
    "regulators",
]

node_d = dict()  # don't want duplicate node entries, therefore going to use dictionary test enforce uniqueness
for _, m in targ_mut_df.iterrows():
    for c in FEAT_COLS:
        for f in m[c]:
            
            mut_cnt = 0
            name = f["name"]
            if c == "genomic features":
                mut_cnt = geno_feat_tot_mut_cnts[f["RegulonDB ID"]]
            if c == "operons":
                mut_cnt = op_tot_mut_cnts[f["RegulonDB ID"]]
                name += " operon"
            if c == "regulators":
                mut_cnt = reg_tot_mut_cnts[f["name"]]
                name += " regulon"
                
            # The id need to match what's in the link dict
            id_s = name  # Used for high-level functions (regs, pwys, etc.)
            if "RegulonDB ID" in f.keys():
                
                id_s = f["RegulonDB ID"]
                if c == "operons": id_s += " operon"  # though don't need to differentiate, like with geno and gen, easier to debug
                if c == "regulators": id_s += " regulon"  # though don't need to differentiate, like with geno and gen, easier to debug
                
            if id_s not in node_d.keys():
                node_d[id_s] = {"id": id_s, "name": str(mut_cnt) + " " + name, "count":mut_cnt}

nodes = []
for k, v in node_d.items():
    nodes.append({"data": v})

In [7]:
# Imodulons currently link to operon names rather than RegulonDB values

operon_df = pd.read_csv("../data/RegulonDB10/operon.txt", sep="\t", comment='#', header=None)
operon_df.columns = [
    "OPERON_ID",
    "OPERON_NAME",
    "FIRSTGENEPOSLEFT",
    "LASTGENEPOSRIGHT",
    "REGULATIONPOSLEFT",
    "REGULATIONPOSRIGHT",
    "OPERON_STRAND",
    "OPERON_INTERNAL_COMMENT",
    "KEY_ID_ORG"]
operon_df["range"] = operon_df.apply(lambda row: (row["REGULATIONPOSLEFT"], row["REGULATIONPOSRIGHT"]), axis=1)
operon_df = operon_df.set_index("OPERON_NAME", drop=True)
operon_df.head()

,OPERON_ID,FIRSTGENEPOSLEFT,LASTGENEPOSRIGHT,REGULATIONPOSLEFT,REGULATIONPOSRIGHT,OPERON_STRAND,OPERON_INTERNAL_COMMENT,KEY_ID_ORG,range
OPERON_NAME,,,,,,,,,
rob,ECK120011191,4634441,4635310,4634441,4635382,reverse,NaN,ECK12,"(4634441, 4635382)"
argP,ECK120011203,3059753,3060646,3059684,3060680,forward,NaN,ECK12,"(3059684, 3060680)"
lrhA,ECK120011329,2405703,2406641,2405703,2406866,reverse,NaN,ECK12,"(2405703, 2406866)"
rtcR,ECK120011490,3558268,3559866,3558268,3559866,forward,NaN,ECK12,"(3558268, 3559866)"
lrp,ECK120011629,932595,933089,932265,933089,forward,NaN,ECK12,"(932265, 933089)"


In [9]:
# The below code links genomic features directly to operons, skipping genetic features
FEAT_LINK_COLS = [
    'operon links',
    'regulator links',
]


edge_d = dict()  # The issue here is I don't want duplicate edges within the dataset; therefore using a dict check for duplicates

for _, m in targ_mut_df.iterrows():            
    for c in FEAT_LINK_COLS:
        
        link_d = m[c]
        
        if c == "operon links":
            for k, l in link_d.items():
                
                for v in set(l):  # Must only look at unique entries in the case of overlap features
                    target_id = k + " operon"
                    geno_sources = m["genetic feature links"][v]
                    
                    for source_id in geno_sources:
                        id_s = target_id + '-' + source_id
                        if id_s not in edge_d.keys():
                            edge_d[id_s] = {"id": id_s, 'target': target_id, "source": source_id, "count": 1}
                        else:
                            edge_d[id_s]["count"] += 1  # Figure out where to put this count line to fix bug
        
        if c == "regulator links":
            for k, l in link_d.items():
                for v in l:

                    target_id = k + " regulon"
                    source_id = v + " operon"
                    
                    id_s = target_id + '-' + source_id
                    if id_s not in edge_d.keys():
                        edge_d[id_s] = {"id": id_s, 'target': target_id, "source": source_id, "count": 1}
                    else:
                        edge_d[id_s]["count"] += 1
                    
edges = []
for k, v in edge_d.items():
    edges.append({"data":v})

In [10]:
mutnet_data = dict()
mutnet_data["nodes"] = nodes
mutnet_data["edges"] = edges

In [11]:
import json
with open('glycerol_mutnet.json', 'w') as outfile:
    json.dump(mutnet_data, outfile)